In [ ]:
### Import Libraries.

library("Seurat")
library("SeuratWrappers")
library("monocle3")
library("ggplot2")
library("dplyr")
library("ggsci")
library("Matrix")
library("readxl")
library("writexl")
library("qs")

In [ ]:
### Load Data.

setwd("/folder/")
Seurat <- qread("Seurat.rds")

In [ ]:
### Define Identity Class.

Idents(Seurat) <- "Cluster_Column"

In [ ]:
### Seurat to Monocle3 CellDataSet.

cds <- as.cell_data_set(Seurat)

In [ ]:
### Cluster Cells and Learn Principal Graph.

cds <- cluster_cells(cds, reduction_method = "UMAP")
cds <- learn_graph(cds, use_partition = TRUE, learn_graph_control = list(minimal_branch_len = 10))

In [ ]:
### Plot.

plot_cells(
  cds,
  color_cells_by = "Cluster_Column",
  label_groups_by_cluster = FALSE,
  label_principal_points = TRUE,
  label_branch_points = FALSE,
  label_roots = FALSE,
  label_leaves = FALSE
)

In [ ]:
### Define Root Principal Nodes.

cds <- order_cells(cds, reduction_method = "UMAP", root_pr_nodes = c("node_1", "node_2"))

In [ ]:
### Plot.

plot_cells(cds, color_cells_by = "pseudotime", show_trajectory_graph = FALSE)

In [ ]:
### Extract Pseudotime and Add to Metadata.

cds$pseudotime <- pseudotime(cds)
cds$Cell_ID <- rownames(colData(cds))

In [ ]:
### Summarize Pseudotime per Cluster_Column

metadata <- as.data.frame(colData(cds))
pseudotime_means <- metadata %>%
  group_by(Cluster_Column) %>%
  summarise(mean_pseudotime = mean(pseudotime, na.rm = TRUE))

In [ ]:
### Plot Pseudotime Distribution Across Clusters.

metadata %>%
  ggplot(aes(x = monocle3_pseudotime, y = reorder(Cluster_Column, monocle3_pseudotime, median))) +
  geom_boxplot(fill = "#4DBBD5FF") +
  theme_minimal() +
  labs(x = "Pseudotime", y = "Cluster_Column")